## SQL Cleaning Checklist
All functions using Standard SQL unless otherwise notated 

1. Column Profiling
2. Missing Data
3. Duplicate Data
4. Correct Data Types
5. Numeric Data
6. Text Data
7. Date and Time Data
8. Categorical Data
9. Verify

Column Profiling

In [ ]:
-- Numeric Column
SELECT 
COUNT(*) AS row_count, COUNT(product_id) AS non_null_count, 
(COUNT(*) - COUNT(product_id)) AS null_count, COUNT(distinct product_id) AS distinct_count, 
MIN(product_id) AS column_min, MAX(product_id) AS column_max, ROUND(AVG(product_id),2) AS column_mean
FROM your_table_name
----------------------------------------------------------------------------------------
-- VARCHAR Column
SELECT 
COUNT(*) AS row_count, COUNT(product_name) AS non_null_count, 
(COUNT(*) - COUNT(product_name)) AS null_count, COUNT(distinct product_name) AS distinct_count, 
MIN(CHAR_LENGTH(product_name)) AS column_len_min, MAX(CHAR_LENGTH(product_name)) AS column_len_max 
FROM your_table_name

Missing Data

In [ ]:
-- Identify NULLs (from above)
SELECT 
COUNT(*) AS row_count, COUNT(product_name) AS non_null_count, (COUNT(*) - COUNT(product_name)) AS null_count
FROM your_table_name
----------------------------------------------------------------------------------------
-- Impute NULLs with default value, similar imputation with AVERAGE(), median, or mode
SELECT
    COALESCE(string_column, 'N/A') AS cleaned_string_column,
    COALESCE(numeric_column, 0) AS cleaned_numeric_column,
    COALESCE(boolean_column, FALSE) AS cleaned_boolean_column,
    COALESCE(date_column, PARSE_DATE('%Y-%m-%d', '1900-01-01')) AS cleaned_date_column, -- PARSE_DATE is a non-standard SQL function (e.g., BigQuery)
FROM
    your_table_name;
----------------------------------------------------------------------------------------
-- Flag a column has NULL value by creating a separate binary flag column
SELECT 
    column_1,
    CASE WHEN column_1 IS NULL THEN 0 ELSE 1 END AS null_flag
FROM   
    your_table_name;
----------------------------------------------------------------------------------------
-- Remove Rows with Excessive NULLs (if imputation is not suitable)
SELECT *
FROM
    your_table_name
WHERE
    column_name_1 IS NOT NULL
    AND column_name_2 IS NOT NULL
    AND column_name_3 IS NOT NULL; 

Duplicate Data

In [ ]:
/*"Unique": That row or combination of columns only appearing once in the dataset. This implies there are no other duplicate instances of that specific row or combination.
"Distinct": One value from each group, including truly unique values. This means if a combination appears 5 times, you get one instance of it. If it appears 1 time (truly unique), you also get that one instance.*/

/* 
Option 1 - Unique and Distinct row/combinations with GROUP BY 
- [PREMIUM] Value Distribution AND
- [PREMIUM] Truly unique rows, appearing only once in data set AND
- Be able to customize "uniqueness" and "distinctness" by selecting combinations OR
- Entirely distinct Rows 
*/

SELECT
    colA,
    colB,
    colC,
    COUNT(*) AS group_count
FROM
    your_table_name
GROUP BY
    colA,
    colB,
    colC
-- HAVING COUNT(*) = 1
ORDER BY group_count ASC;
----------------------------------------------------------------------------------------
/* 
Option 2 - Ordered, distinct row/combinations with ROW_NUMBER()
- [PREMIUM] Be able to select all columns from the table AND
- [PREMIUM] Be able to choose which row to keep if there are duplicates such as the most recent occurrence AND 
- Be able to customize "distinctness" by selecting distinct combinations OR
- Entirely distinct rows 
*/

WITH subquery_alias AS 
    (
        SELECT
            *, -- Or list all relevant columns
            ROW_NUMBER() OVER (PARTITION BY colA, colB, colC /* ... columns defining "distinctness" */ ORDER BY (colA DESC)) as rn
            -- ORDER BY clause: pick most recent, last, etc with ASC/DESC
        FROM
            your_table_name
    )
SELECT
    colA,
    colB,
    colC
FROM subquery_alias
WHERE
    rn = 1; -- > 1 for just duplicates

----------------------------------------------------------------------------------------
/* 
Option 3 - Identify dupes while retaining all rows with COUNT(*) OVER(PARTITION BY...) 
- [PREMIUM] Calculations between rows and groups (like % frequency)
- [PREMIUM] Be able to filter rows based on group properties
- [PREMIUM] Be able to select all columns from the table AND
- Be able to customize "uniqueness" by selecting combinations
- Add ORDER BY to get running counts/other aggregates (limitation: groups ties together)
*/

WITH SubQueryAlias AS
(
    SELECT
        *,
        COUNT(*) OVER(PARTITION BY colA) AS colA_count,
        COUNT(*) OVER(PARTITION BY colB) AS colB_count
    FROM your_table_name
) 
SELECT 
    colA, colB, colC
FROM
    SubQueryAlias
WHERE 
    colA_2015_count > 1 AND colB_count = 1;


Data Type Conversion

In [ ]:
SELECT
    CAST(string_as_int_column AS INTEGER) AS converted_int_column,
    CAST(string_as_float_column AS FLOAT) AS converted_float_column,
    CAST(numeric_as_boolean_column AS BOOLEAN) AS converted_boolean_column, -- 0:FALSE, 1:TRUE.
    SAFE.PARSE_DATE('%Y-%m-%d', string_as_date_column) AS converted_date_column, -- SAFE.PARSE_DATE: GoogleSQL
    SAFE.PARSE_TIMESTAMP('%Y-%m-%d %H:%M:%S', string_as_timestamp_column) AS converted_timestamp_column -- SAFE.PARSE_TIMESTAMP: GoogleSQL
FROM
    your_table_name;

Numeric Data

In [ ]:
-- Assign NULL to negative values that shouldn't be negative, like age
-- Assign a floor or ceiling value past a threshold. The threshold could be a percentile using PERCENTILE_CONT()
SELECT
    CASE
        WHEN age_column < 0 THEN NULL ELSE age_column END AS cleaned_age_column,
FROM
    your_table_name;

----------------------------------------------------------------------------------------
-- Removing Outliers above or below percentiles
WITH PercentileBounds AS 
(
    SELECT
        PERCENTILE_CONT(0.05) WITHIN GROUP (ORDER BY numeric_column) OVER () AS lower_bound,
        PERCENTILE_CONT(0.95) WITHIN GROUP (ORDER BY numeric_column) OVER () AS upper_bound
    FROM
        your_table_name
    WHERE
        numeric_column IS NOT NULL -- Percentile_CONT typically ignore NULLs, explicit filtering anyway                                 
)
SELECT
    t.id,
    t.numeric_column
FROM
    your_table_name AS t 
CROSS JOIN 
    PercentileBounds AS pb -- join single-row result to all rows of main table with cross-join
WHERE
    t.numeric_column >= pb.lower_bound
    AND t.numeric_column <= pb.upper_bound;

Text Data

In [ ]:
-- Remove leading, trailing, or excessive internal whitespace
SELECT
    TRIM(string_column) AS trimmed_string, -- leading and trailing whitespace
    REGEXP_REPLACE(string_column, r'\s+', ' ') AS single_space_string -- Replace multiple spaces \s+ with single, not used in T-SQL
FROM
    your_table_name;
----------------------------------------------------------------------------------------
-- Standardize Case into Upper/Lower/Proper 
SELECT
    UPPER(string_column) AS uppercase_string,
    LOWER(string_column) AS lowercase_string,
    -- For proper case (first letter capitalized, rest lowercase)
    CONCAT(UPPER(SUBSTR(string_column, 1, 1)), LOWER(SUBSTR(string_column, 2))) AS proper_case_string -- SUBSTR is a non-standard SQL function (e.g., MySQL, PostgreSQL, BigQuery)
FROM
    your_table_name;
----------------------------------------------------------------------------------------
-- Remove Non-Alphanumeric Characters, Keep only letters, numbers, and spaces
SELECT
    REGEXP_REPLACE(string_column, r'[^a-zA-Z0-9\s]', '') AS alphanumeric_only_string, -- REGEXP_REPLACE is a non-standard SQL function (e.g., PostgreSQL, BigQuery), english characters only
    
    REGEXP_REPLACE(string_column, r'[^\p{L}\p{N}\s]', '') AS unicode_alphanumeric_only_string -- REGEXP_REPLACE and \p{L}, \p{N} are non-standard SQL functions/regex features (e.g., BigQuery), international character friendly
FROM
    your_table_name;
----------------------------------------------------------------------------------------

Date and Time Data

In [ ]:
-- Handle multiple date formats using CASE
-- SAFE.PARSE_DATE is a non-standard SQL function (e.g., BigQuery)
-- ISO 8601 Date Only YYYY-MM-DD

SELECT
    CASE
        -- Format 1: YYYY-MM-DD
        WHEN SAFE.PARSE_DATE('%Y-%m-%d', mixed_date_string) IS NOT NULL
            THEN SAFE.PARSE_DATE('%Y-%m-%d', mixed_date_string)
        -- Format 2: MM-DD-YYYY
        WHEN SAFE.PARSE_DATE('%m-%d-%Y', mixed_date_string) IS NOT NULL
            THEN SAFE.PARSE_DATE('%m-%d-%Y', mixed_date_string)
        -- Format 3: DD-MM-YYYY
        WHEN SAFE.PARSE_DATE('%d-%m-%Y', mixed_date_string) IS NOT NULL
            THEN SAFE.PARSE_DATE('%d-%m-%Y', mixed_date_string)
        -- Format 4: YYYY/MM/DD
        WHEN SAFE.PARSE_DATE('%Y/%m/%d', mixed_date_string) IS NOT NULL
            THEN SAFE.PARSE_DATE('%Y/%m/%d', mixed_date_string)
        -- Format 5: MM/DD/YYYY
        WHEN SAFE.PARSE_DATE('%m/%d/%Y', mixed_date_string) IS NOT NULL
            THEN SAFE.PARSE_DATE('%m/%d/%Y', mixed_date_string)
        -- Format 6: DD/MM/YYYY
        WHEN SAFE.PARSE_DATE('%d/%m/%Y', mixed_date_string) IS NOT NULL
            THEN SAFE.PARSE_DATE('%d/%m/%Y', mixed_date_string)
        ELSE NULL -- If none of the above formats match
    END AS robust_date_parsing
FROM
    your_table_name;
----------------------------------------------------------------------------------------
-- Handle multiple timestamp formats using CASE
-- SAFE.PARSE_TIMESTAMP is a non-standard SQL function (e.g., BigQuery)
-- ISO 8601 Timestamp Only YYYY-MM-DD HH:MM:SS

SELECT
    CASE
        -- Format 1: YYYY-MM-DD HH:MI:SS
        WHEN SAFE.PARSE_TIMESTAMP('%Y-%m-%d %H:%M:%S', mixed_timestamp_string) IS NOT NULL
            THEN SAFE.PARSE_TIMESTAMP('%Y-%m-%d %H:%M:%S', mixed_timestamp_string)
        -- Format 2: MM-DD-YYYY HH:MI:SS
        WHEN SAFE.PARSE_TIMESTAMP('%m-%d-%Y %H:%M:%S', mixed_timestamp_string) IS NOT NULL
            THEN SAFE.PARSE_TIMESTAMP('%m-%d-%Y %H:%M:%S', mixed_timestamp_string)
        -- Format 3: DD-MM-YYYY HH:MI:SS
        WHEN SAFE.PARSE_TIMESTAMP('%d-%m-%Y %H:%M:%S', mixed_timestamp_string) IS NOT NULL
            THEN SAFE.PARSE_TIMESTAMP('%d-%m-%Y %H:%M:%S', mixed_timestamp_string)
        -- Format 4: YYYY/MM/DD HH:MI:SS
        WHEN SAFE.PARSE_TIMESTAMP('%Y/%m/%d %H:%M:%S', mixed_timestamp_string) IS NOT NULL
            THEN SAFE.PARSE_TIMESTAMP('%Y/%m/%d %H:%M:%S', mixed_timestamp_string)
        -- Format 5: MM/DD/YYYY HH:MI:SS
        WHEN SAFE.PARSE_TIMESTAMP('%m/%d/%Y %H:%M:%S', mixed_timestamp_string) IS NOT NULL
            THEN SAFE.PARSE_TIMESTAMP('%m/%d/%Y %H:%M:%S', mixed_timestamp_string)
        -- Format 6: DD/MM/YYYY HH:MI:SS
        WHEN SAFE.PARSE_TIMESTAMP('%d/%m/%Y %H:%M:%S', mixed_timestamp_string) IS NOT NULL
            THEN SAFE.PARSE_TIMESTAMP('%d/%m/%Y %H:%M:%S', mixed_timestamp_string)
        ELSE NULL -- If none of the above formats match
    END AS robust_timestamp_parsing
FROM
    your_table_name;

----------------------------------------------------------------------------------------
-- Extracting parts from a DATE
SELECT
    EXTRACT(YEAR FROM date_column) AS invoice_year,
    EXTRACT(MONTH FROM date_column) AS invoice_month,
    EXTRACT(DAY FROM date_column) AS invoice_day,
    EXTRACT(WEEK FROM date_column) AS week_of_year,
    EXTRACT(QUARTER FROM date_column) AS quarter_of_year
FROM
    my_invoices;

----------------------------------------------------------------------------------------
-- Extracting parts from a TIMESTAMP
SELECT
    EXTRACT(YEAR FROM timestamp_column) AS event_year,
    EXTRACT(MONTH FROM timestamp_column) AS event_month,
    EXTRACT(DAY FROM timestamp_column) AS event_day,
    EXTRACT(HOUR FROM timestamp_column) AS event_hour,
    EXTRACT(MINUTE FROM timestamp_column) AS event_minute,
    EXTRACT(SECOND FROM timestamp_column) AS event_second_with_fractions,
    EXTRACT(WEEK FROM timestamp_column) AS event_week,
    EXTRACT(QUARTER FROM timestamp_column) AS event_quarter
FROM
    my_events;

Categorical Data

In [ ]:
-- Correct Inconsistent Categorical Values (Mapping/Standardization)
SELECT
    CASE
        WHEN country_column IN ('USA', 'U.S.A.', 'United States') THEN 'United States'
        WHEN country_column IN ('UK', 'U.K.', 'United Kingdom') THEN 'United Kingdom'
        ELSE country_column
    END AS standardized_country
FROM
    your_table_name;
----------------------------------------------------------------------------------------
-- Ensure boolean values are consistently represented (e.g., TRUE/FALSE, 1/0)
SELECT
    id,
    CASE
        WHEN is_active_column IN ('true', '1', 'yes') THEN TRUE
        WHEN is_active_column IN ('false', '0', 'no') THEN FALSE
        ELSE NULL -- Or a default boolean value
    END AS standardized_is_active
FROM
    your_table_name;


Verify

In [ ]:
-- Ranges of numeric or data values 
SELECT
    *
FROM
    your_table_name
WHERE
    age_column BETWEEN 0 AND 120 -- Age must be between 0 and 120
    AND price_column >= 0; -- Price cannot be negative
----------------------------------------------------------------------------------------
-- Unique & Distinct Values 
SELECT
    colA,
    colB,
    colC,
    COUNT(*) AS group_count
FROM
    your_table_name
GROUP BY
    colA,
    colB,
    colC
-- HAVING COUNT(*) = 1
ORDER BY group_count ASC;
----------------------------------------------------------------------------------------
-- Referential Integrity: ensure every foreign key record has a matching primary key in the primary table
SELECT
    ct.*
FROM
    child_table ct
LEFT JOIN
    parent_table pt ON ct.parent_id = pt.id
WHERE
    pt.id IS NULL;
----------------------------------------------------------------------------------------
--Logic: birth_date < admission_date < discharge_date, etc
SELECT
    *
FROM
    your_table_name
WHERE
    start_date_column > end_date_column; -- Identify records where end date is before start date